# `gmxapi` Practice
In this notebook, our goal is to use `gmxapi` to run a standard MD simulation, an expanded ensemble simulaiton, and a 1D alchemical metadynamics simulation. Note that before running this notebook, one should first load in GROMACS and activate corresponding virtual environment as needed. Delete the folders like `standard_MD`, `EE`, `lambda_MetaD`, and `gmxapi.commadline.cli*_i0` if they exist. These folders will be generated later along the way. Notably, the examples set up in this notebooks are demonstrating the usage of `gmxapi` with GROMACS only running with thread-MPI. 

In [1]:
import sys
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')  # might not be necessary; change the path as needed
import gmxapi as gmx

## Section 1. Running a standard MD simulation using `gmxapi`

To get started, we set up the directories as below:

In [2]:
%%bash
mkdir standard_MD && cd standard_MD
cp -r ../../inputs/* .
mkdir Box Sol EM Equil MD Topology
mv *top Topology/.
cd EM && cp ../mdp_files/em.mdp .
cd ../Equil && mkdir NVT && cd NVT && cp ../../mdp_files/nvt_equil.mdp .
cd ../ && mkdir NPT && cd NPT && cp ../../mdp_files/npt_equil.mdp .
cd ../../MD && cp ../mdp_files/md.mdp .

### 1. Construct the simulation box
Here we use `sys2_init.gro` as the input file, which is the same as `simple_system2.gro` except that the box vector was removed. Normally, the GROMACS command would be as follows if the working directory is the path of this notebook:
```
gmx editconf -f standard_MD/sys2_init.gro -o standard_MD/Box/sys2_box.gro -bt dodecahedron -d 1 -c
```
Using `gmxapi`, we first create an object using `gmx.commandline_operation` and run `object.run()`:

In [3]:
box = gmx.commandline_operation('gmx',
                               arguments=['editconf', '-bt', 'dodecahedron', '-d', '1', '-c'],
                               input_files={'-f': '../standard_MD/sys2_init.gro'},
                               output_files={'-o': '../standard_MD/Box/sys2_box.gro'})
box.run()

Note that `box.run()` will create a folder `gmxapi.commandline.cli*_i0` and use it as the working directory, so the path to the input files and output files should be `../standard_MD/sys2_init.gro`. Additionally, if `output_files` is not specified, then the default file names will be used and the outputs will be samed in the newly created folder `gmxapi.commandline.cli*_i0`. Here, we specify the path of the output so that the folder `gmxapi.commandline.cli*_i0` will be empty after the command is finished.

To check if the command was executed successfully, we can check `returncode` as follows. Additionally, we can check `stdout` and `stderr` to get more information about the output.

In [4]:
print(f'Return code of the process: {box.output.returncode.result()}\n')
print(f'STDERR of the process:\n\n {box.output.stderr.result()}\n')
print(f'STDOUT of the process:\n\n {box.output.stdout.result()}\n')

Return code of the process: 0

STDERR of the process:

               :-) GROMACS - gmx editconf, 2021.4-plumed-2.7.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
      

We can also check the filename of the output and the flag where it was specified as follows:

In [5]:
print(box.output.file.result())

{'-o': '/home/wei-tse/Documents/gmxapi_practice/standard_MD/Box/sys2_box.gro'}


### 2. Solvate the system

Again, to solvate the system, the original GROMACS command is as follows:
```
gmx solvate -cp standard_MD/Box/sys2_box.gro -p standard_MD/Topology/sys2.top -o sys2_sol.gro -cs
```
Therefore, we can execute the command as follows. Note that now we could either explicitly specify the output file of the previous command or use the `box` object to retrieve the output filename in the last step. Also note that `box.output.file['-o']` is a `gmxapi.operation.Future` object, and `box.output.file['-o'].result()`.
```
>>> print(box.output.file['-o'])
<Future: name='file', description=ResultDescription(dtype=None, width=1)>
```

In [6]:
solvate = gmx.commandline_operation('gmx',
                                   arguments=['solvate', '-cs'],
                                   input_files={
                                       '-cp': box.output.file['-o'],
                                       '-p': '../standard_MD/Topology/sys2.top'},
                                   output_files={'-o': '../standard_MD/Sol/sys2_sol.gro'})
solvate.run()

In [7]:
if solvate.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {solvate.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


### 3. Energy minimization

Again, we run the following GROMACS commands using `gmxapi`:
```
gmx grompp -f standard_MD/EM/em.mdp -c standard_MD/Sol/sys2_sol.gro -p standard_MD/Topology/sys2.top -o standard_MD/EM/sys2_em.tpr
gmx mdrun -s standard_MD/EM/sys2_em.tpr -c standard_MD/EM/sys2_em.gro -g standard_MD/EM/em.log -e standard_MD/EM/em.edr
```

In [8]:
grompp_em = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/EM/em.mdp',
                                         '-c': '../standard_MD/Sol/sys2_sol.gro',
                                         '-p': '../standard_MD/Topology/sys2.top'},
                                     output_files={'-o': '../standard_MD/EM/sys2_em.tpr'})
grompp_em.run()
if grompp_em.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {grompp_em.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


There are two ways to launch an `mdrun` command. The first way is to use `commandline_operation` as before. This time we demonstrate how one could use `grompp.output.file['-o']` as an input here. 

In [9]:
em = gmx.commandline_operation('gmx',
                              arguments=['mdrun'],
                              input_files={'-s': grompp_em.output.file['-o']},
                              output_files={
                                  '-c': '../standard_MD/EM/sys2_em.gro',
                                  '-e': '../standard_MD/EM/em.edr',
                                  '-g': '../standard_MD/EM/em.log',
                                  '-o': '../standard_MD/EM/em.trr'})
em.run()
if em.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {em.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')

The process was executed successfully.


Another way to run a `mdrun` command using `gmxapi` is to use `gmx.mdrun`:
```
em_input = gmx.read_tpr(grompp_em.output.file['-o'])
# em_modified = gmx.modify_input(input=em_input, parameters={'nsteps': 100})  # modify the parameters as needed, only work with gmx.mdrun (not gmx.commandline_operation)
em = gmx.mdrun(input=em_input)
em.run()
```
Or simply
```
em = gmx.mdrun(input=grompp_em.output.file['-o'])
em.run()
```
Note:
  - The argument `input` should be the tpr file and `gmx.mdrun` creates a folder named as `mdrun_*_i0_0/`.
  - It doesn't seem that we can specify the output file names though ...
  - In this case, we can't retrieve the return code, stdout, or stderr as before using a line like `em.output.returncode` since `em` is not the same type of object if `gmx.mdrun` is used.

### 4. Equilibration

To run NVT and then NPT equilibration using GROMACS, we use the following commands:
```
gmx grompp -f standard_MD/Equil/NVT/nvt_equil.mdp -c standard_MD/EM/sys2_em.gro -p standard_MD/Topology/sys2.top -o standard_MD/Equil/NVT/sys2_equil.tpr
gmx mdrun -s standard_MD/Equil/NVT/sys2_equil.tpr -c standard_MD/Equil/NVT/sys2_equil.gro -g standard_MD/Equil/NVT/equil.log -e standard_MD/Equil/NVT/equil.edr
gmx grompp -f npt_equil.mdp -c ../NVT/sys2_equil.gro -t ../NVT/state.cpt -p ../../Topology/sys2.top -o sys2_equil.tpr
gmx mdrun -s sys2_equil.tpr -c sys2_equil.gro -g equil.log -e equil.edr
```
Below we first define a simple function to deal with `stderr` first, and then we execute the commands above using `gmxapi`. Note that the equilibration length was very short because this notebook is just for demonstration purpose. Also note that the will be an `xtc` file output by `mdrun` but it will be saved in the newly-created folder since we don't specify it in `output_files`.

In [10]:
def gmx_output(obj):
    if obj.output.returncode.result() != 0:
        print(f'STDERR of the process:\n\n {obj.output.stderr.result()}\n')
    else:
        print('The process was executed successfully.')

In [11]:
print('Performing NVT equilibration ...')
grompp_nvt = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/Equil/NVT/nvt_equil.mdp',
                                         '-c': '../standard_MD/EM/sys2_em.gro',
                                         '-p': '../standard_MD/Topology/sys2.top'},
                                     output_files={'-o': '../standard_MD/Equil/NVT/sys2_equil.tpr'})
grompp_nvt.run()
gmx_output(grompp_nvt)
    
nvt = gmx.commandline_operation('gmx',
                               arguments=['mdrun'],
                               input_files={'-s': grompp_nvt.output.file['-o']},
                               output_files={
                                   '-c': '../standard_MD/Equil/NVT/sys2_equil.gro',
                                   '-e': '../standard_MD/Equil/NVT/equil.edr',
                                   '-g': '../standard_MD/Equil/NVT/equil.log',
                                   '-o': '../standard_MD/Equil/NVT/equil.trr',
                                   '-cpo': '../standard_MD/Equil/NVT/state.cpt'})
nvt.run()
gmx_output(nvt)
    
print('\nPerforming NPT equilibration ...')
grompp_npt = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/Equil/NPT/npt_equil.mdp',
                                         '-c': '../standard_MD/Equil/NVT/sys2_equil.gro',
                                         '-p': '../standard_MD/Topology/sys2.top',
                                         '-t': '../standard_MD/Equil/NVT/state.cpt'},
                                     output_files={'-o': '../standard_MD/Equil/NPT/sys2_equil.tpr'})
grompp_npt.run()
gmx_output(grompp_npt)
    
npt = gmx.commandline_operation('gmx',
                               arguments=['mdrun'],
                               input_files={'-s': grompp_npt.output.file['-o']},
                               output_files={
                                   '-c': '../standard_MD/Equil/NPT/sys2_equil.gro',
                                   '-e': '../standard_MD/Equil/NPT/equil.edr',
                                   '-g': '../standard_MD/Equil/NPT/equil.log',
                                   '-o': '../standard_MD/Equil/NPT/equil.trr',
                                   '-cpo': '../standard_MD/Equil/NPT/state.cpt'})
npt.run()
gmx_output(npt)

Performing NVT equilibration ...
The process was executed successfully.
The process was executed successfully.

Performing NPT equilibration ...
The process was executed successfully.
The process was executed successfully.


### 5. Launch the simulation

Lastly, we use the `gmxapi` to run the following commands:
```
gmx grompp -f standard_MD/MD/md.mdp -c standard_MD/Equil/NPT/sys2_equil.gro -p ../Topology/sys2.top -o sys2_md.tpr -t ../Equil/NPT/state.cpt
gmx mdrun -s sys2_md.tpr -x sys2_md.xtc -c sys2_md.gro -g md.log -e md.edr
```

In [12]:
grompp_md = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../standard_MD/MD/md.mdp',
                                         '-c': '../standard_MD/Equil/NPT/sys2_equil.gro',
                                         '-p': '../standard_MD/Topology/sys2.top',
                                         '-t': '../standard_MD/Equil/NPT/state.cpt'},
                                     output_files={'-o': '../standard_MD/MD/sys2_md.tpr'})
grompp_md.run()
gmx_output(grompp_md)
    
md = gmx.commandline_operation('gmx',
                              arguments=['mdrun'],
                              input_files={'-s': grompp_npt.output.file['-o']},
                              output_files={
                                  '-c': '../standard_MD/MD/sys2_md.gro',
                                  '-e': '../standard_MD/MD/md.edr',
                                  '-g': '../standard_MD/MD/md.log',
                                  '-o': '../standard_MD/MD/md.trr'})
md.run()
gmx_output(md)

The process was executed successfully.
The process was executed successfully.


## Section 2. Running expanded ensemble simulations using `gmxapi`

Here we take the output of the standard MD done in Section 1 as the input of expanded ensemble. The way we run exapnded ensemble simulations using `gmxapi` is pretty much the same as what we've done in Section 1. Here is just a section showing that `gmxapi` can also run advanced sampling methods.

To make the demonstration a little more interesting, instead of running just 1 expanded ensemble, the goal of this section is to run 4 expanded ensemble at the same time to show `gmxapi`'s ability to run an emsemble of simulation. Note that in this case, the 4 simulations are independent of each other and this can only be done in `gmx.mdrun`, not `gmx.commandline_operation`.

Before we get to running 4 simulations using `gmxapi`, let's first try running only 1 expanded ensemble simulation using `gmxapi`.

### 1. Running 1 expanded ensemble simulation using `gmxapi`

Again, here we first set up relevant directory strucutres.

In [13]:
%%bash 
mkdir EE && cd EE 
mkdir case_1 case_2 && cd case_1
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [14]:
grompp_EE = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../EE/case_1/expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': '../EE/case_1/sys2.top'},
                                     output_files={'-o': '../EE/case_1/sys2_EE.tpr'})
grompp_EE.run()
gmx_output(grompp_EE)

The process was executed successfully.


Note that it seems that running expanded ensemble from Jupyter notebook can easily make the notebook dead. So we write the following script `EE.py` (saved in the folder `EE/case_1`) that will be executed from the terminal.
```python
import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

EE = gmx.mdrun('sys2_EE.tpr')
EE.run()
```

In [15]:
%%bash 
cd EE/case_1 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

EE = gmx.mdrun('sys2_EE.tpr')
EE.run()"
    
echo "${code}" >> EE.py

python EE.py

Reading file sys2_EE.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file /home/wei-tse/Documents/gmxapi_practice/EE/case_1/mdrun_0_i0_0/topol.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file /home/wei-tse/Documents/gmxapi_practice/EE/case_1/mdrun_0_i0_0/topol.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Changing nstlist from 10 to 40, rlist from 0.9 to 0.998

Using 1 MPI thread
Using 16 OpenMP threads 

starting mdrun 'Generic title in water'
10000 steps,     20.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:      120.009        7.501     1600.0
                 (ns/day)    (hour/ns)
Performance:      230.403        0.104


### 2. Running 4 expanded ensemble simulations using `gmxapi`

First of all, we set up directory structures and generate the `.tpr` files.

In [16]:
%%bash 
cd EE/case_2 
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [17]:
for i in range(4):
    grompp_EE = gmx.commandline_operation('gmx', 
                                         arguments=['grompp'],
                                         input_files={
                                             '-f': '../EE/case_2/expanded.mdp',
                                             '-c': '../standard_MD/MD/sys2_md.gro',
                                             '-p': '../EE/case_2/sys2.top'},
                                         output_files={'-o': f'../EE/case_2/sys2_EE_{i}.tpr'})
    grompp_EE.run()
    gmx_output(grompp_EE)

The process was executed successfully.
The process was executed successfully.
The process was executed successfully.
The process was executed successfully.


Similarly, in this case we will run a Python scrtip from the terminal rather than executing in a notebook cell. What's different is that this time we need MPI (e.g. `mpiexec`, `mpirun`, etc.) to run the simulations in parallel (but not communicating in our case). Here we are just using GROMACS with thread-MPI and there is no `mpirun` available, so we use `mpiexec` and `mpi4py` below.

Below is the script that can be used to run multiple expanded ensemble simulations:
```python
import sys
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

tpr_list = [f'../EE/sys2_EE_{i}.tpr' for i in range(4)]
EE_inputs = gmx.read_tpr(tpr_list)
EE = gmx.mdrun(EE_inputs)
EE.run()
```

In [18]:
%%bash 
cd EE/case_2 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

tpr_list = [f'sys2_EE_{i}.tpr' for i in range(4)]
EE_inputs = gmx.read_tpr(tpr_list)
EE = gmx.mdrun(EE_inputs)
EE.run()"
    
echo "${code}" >> EE.py

mpiexec -n 4 python3.7 -m mpi4py EE.py    # mpi4py only available in python3.7 in our case

Reading file sys2_EE_0.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_0.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_0.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_0.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_1.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_1.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_1.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_1.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_2.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_2.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_2.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_2.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_3.tpr, VERSION 2021.4-plumed-2.7.3 (single precision)
Reading file sys2_EE_3.tp

Upon execution, there will be 4 folders in `case_2` named in the form of `mdrun_0_i0_*`, 1 for each simulation. 

Notably, if `mpiexec` is not used (namely, if one simply runs `python EE.py`), the following error would occur:
```python
Traceback (most recent call last):
  File "EE.py", line 8, in <module>
    EE.run()
  File "/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages/gmxapi/simulation/mdrun.py", line 690, in run
    self.__resource_manager.update_output()
  File "/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages/gmxapi/simulation/mdrun.py", line 602, in update_output
    input = LegacyImplementationSubscription(self)
  File "/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages/gmxapi/simulation/mdrun.py", line 245, in __init__
    with scoped_communicator(context_comm, ensemble_width) as ensemble_comm:
  File "/home/wei-tse/anaconda3/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages/gmxapi/simulation/mdrun.py", line 175, in scoped_communicator
    original_comm_size
gmxapi.exceptions.FeatureNotAvailableError: Cannot produce a subcommunicator of size 4 from a communicator of size 1.
```

Notably, the value of `n` should at least be 4 since we have 4 simulations to run. Setting the value of `n` as 4, the performance of each simulation is around 15 ns/day, while the performance of 1 expanded ensemble simulation (using all the 16 cores) was around 240 ns/day. The parallelizing simulations here are not ideal because we ran all the simulations within 1 node. If multiple nodes are available, the scaling should be better. (In our case here, simulations with `-n` being 16 would be even slower.)

## Section 3. Running alchemical metadynamics simulations using `gmxapi`

Lastly, we want to see if `gmxapi` works with PLUMED, or more specifically, if `gmxapi` can run alchemical metadynamics simulations, including 1D and 2D cases. Note that here we are using a certain commit of PLUMED 2.7.3, which is the commit where the issue of alchemical metadynamics had been fixed but had not merged to the branch `v2.8` and changed the way that the mdp file should have been specified. Therefore, in this specific version of PLUMED, a mdp file for alchemical metadynamics would just be the same as the one used in expanded ensemble (weight-equilibrating phase). 

### 1. 1D alchemical metadynamics
Again, we first set up directory structures for a 1D alchemical metadynamics simulation. 

In [19]:
%%bash 
mkdir lambda_MetaD && cd lambda_MetaD 
mkdir 1D_case 2D_case && cd 1D_case
cp ../../../inputs/plumed_1.dat . 
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [20]:
grompp_lambda_1 = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../lambda_MetaD/1D_case/expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': '../lambda_MetaD/1D_case/sys2.top'},
                                     output_files={'-o': '../lambda_MetaD/1D_case/sys2_lambda.tpr'})
grompp_lambda_1.run()
gmx_output(grompp_lambda_1)

The process was executed successfully.


Similarly, we run the simulation by executing a Python script from the terminal. Note that the Python script `lambda_MetaD.py` was executed in the folder `lambda_MetaD`, so the paths of the input and output files in the script below should be changed accordingely.

In [21]:
%%bash 
cd lambda_MetaD/1D_case 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

lambda_MetaD = gmx.commandline_operation('gmx',
                              arguments=['mdrun', '-plumed', '../plumed_1.dat'],
                              input_files={'-s': '../sys2_lambda.tpr'},
                              output_files={
                                  '-c': '../sys2_lambda.gro',
                                  '-e': '../lambda.edr',
                                  '-g': '../lambda.log',
                                  '-o': '../lambda.trr'})
lambda_MetaD.run()

if lambda_MetaD.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {lambda_MetaD.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')"

echo "${code}" >> lambda_MetaD.py

python lambda_MetaD.py

The process was executed successfully.


Nice! So `gmxapi` did work with PLUMED! (It's kind of weird that the bottom of the log file was not printed here though.) Note that since the path of the PLUMED output files can not be specified from the GROMACS command but defined in `plumed.dat`, all the PLUMED outputs will be saved in the folder `gmxapi.commandline.cli0_i0 `.

### 2. 2D alchemical metadynamics

We repeat the protocol above but with a different PLUMED input file. Here we just want to make sure that `gmxapi` also works with 2D cases. 

In [22]:
%%bash 
cd lambda_MetaD/2D_case
cp ../../../inputs/plumed_2.dat . 
cp ../../../inputs/mdp_files/expanded.mdp .
cp ../../standard_MD/Topology/*top .

In [23]:
grompp_lambda_2 = gmx.commandline_operation('gmx', 
                                     arguments=['grompp'],
                                     input_files={
                                         '-f': '../lambda_MetaD/2D_case/expanded.mdp',
                                         '-c': '../standard_MD/MD/sys2_md.gro',
                                         '-p': '../lambda_MetaD/2D_case/sys2.top'},
                                     output_files={'-o': '../lambda_MetaD/2D_case/sys2_lambda.tpr'})
grompp_lambda_2.run()
gmx_output(grompp_lambda_2)

The process was executed successfully.


In [24]:
%%bash 
cd lambda_MetaD/2D_case 

code="import sys 
sys.path.append('/home/wei-tse/gmxapi_21.4/lib/python3.7/site-packages')
import gmxapi as gmx

lambda_MetaD = gmx.commandline_operation('gmx',
                              arguments=['mdrun', '-plumed', '../plumed_2.dat'],
                              input_files={'-s': '../sys2_lambda.tpr'},
                              output_files={
                                  '-c': '../sys2_lambda.gro',
                                  '-e': '../lambda.edr',
                                  '-g': '../lambda.log',
                                  '-o': '../lambda.trr'})
lambda_MetaD.run()

if lambda_MetaD.output.returncode.result() != 0:
    print(f'STDERR of the process:\n\n {lambda_MetaD.output.stderr.result()}\n')
else:
    print('The process was executed successfully.')"

echo "${code}" >> lambda_MetaD.py

python lambda_MetaD.py

The process was executed successfully.
